# Thermoelasticity-based modal damage identification

## Import packages

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pysfmov as sfmov
#import km
import FLife

## Input parameters

### Thermal video

In [2]:
filename = './data/rec.sfmov'
fs = 400

data = sfmov.get_data(filename)
km = 1/ (6 * 2 * 10**(-9)) * 10**-6 # stress amplitude in MPa  # Calibration Factor [Pa^-1]
stress = data * km

location = 38, 78, 5, 5 #vert mode area
interval = [45, 65] # HZ
df_fft = 0.1

### Material

In [3]:
k = 6.51 # slope endurance curve
B = 800.26 # endurance curve 

## Computation

In [23]:
class ThermalData():
    
    
    def __init__(self, stress, location, interval, df_fft = 0.1):
        
        self.stress = stress
        self.location = location
        self.interval = interval
        self.df_fft = df_fft
    
        ds = self.stress - self.stress[0,:,:] 
        N = self.stress.shape[0]
        
        (x, y, w, h) = self.location
        
        fft = (np.fft.rfft(ds, N, axis=0) * 2 / N)[:, y:(y+h), x:(x+w)]
        freq = np.fft.rfftfreq(N, 1/fs)
        
        fft_avg = np.abs(np.mean(fft, axis = (1,2)))
    
        y_peak = np.max(fft_avg[(freq > self.interval[0]) & (freq < self.interval[1])])
        self.x_peak = freq[np.where(fft_avg == y_peak)[0][0]]
    
        self.amplitude = np.mean(fft_avg[(freq >= self.x_peak - self.df_fft) & (freq <= self.x_peak + self.df_fft)])

    
    def get_natural_freq(self):
        
        return self.x_peak
    
    
    def get_life(self, C, k, method = 'modal'):
        
        if method == 'modal':
            C = B**k  # s = B * N**bas or s^k * N = C
            return (C / self.amplitude**k) / self.x_peak 
        
        elif method == 'TovoBenasciutti':
            
            
        


In [24]:
dam = ThermalData(stress, location, interval, df_fft)

In [25]:
dam.get_life(B,k)

1.028096193473253e+18

## Endurance curve

In [ ]:
N = np.linspace(1,10**8,10**8)
s = B * N**bas

plt.semilogx(N,s)
plt.grid()
plt.xlabel('Cycles [Log(N)]')
plt.ylabel('Stress amplitude [MPa]')
plt.title('Endurance curve')

## FFT and Natural frequency


In [ ]:
plt.figure()
plt.semilogy(freq,fft_avg[0,:], c = 'C0')
plt.grid()
plt.scatter(x_peak, y_peak,s = 100, c='C3', marker = 'o')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Amplitude [MPa]')
plt.title('FFT')

In [8]:
import FLife
import numpy as np


dt = 1e-4
x = np.random.normal(scale=100, size=10000)

C = 1.8e+22
k = 7.3

# Spectral data
sd = FLife.SpectralData(input=x, dt=dt)

# Rainflow reference fatigue life
# (do not be confused here, spectral data object also holds the time domain data)
rf = FLife.Rainflow(sd)

# Spectral methods
dirlik = FLife.Dirlik(sd)
tb = FLife.TovoBenasciutti(sd)
print(f'          Rainflow: {rf.get_life(C = C, k=k):4.0f} s')
print(f'            Dirlik: {dirlik.get_life(C = C, k=k):4.0f} s')
print(f'Tovo Benasciutti 2: {tb.get_life(C = C, k=k, method="method 2"):4.0f} s')

          Rainflow:  267 s
            Dirlik:  127 s
Tovo Benasciutti 2:  145 s
